In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import numpy as np
import scipy.io as sio
from tensorflow.keras.utils import to_categorical
from keras.callbacks import ReduceLROnPlateau
import sklearn.metrics
from sklearn.utils import shuffle 
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Reshape
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, Conv3D, MaxPooling3D, BatchNormalization
import matplotlib.pyplot as plt
from keras.models import load_model


from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model

In [2]:
data  = np.load("E:/STUDY/Publications/Thesis/Brain Emotion Detection/Dataset/Practice with datasets/Seed/de_LDS_Extracted/Shuffled X y/shuffled_data.npy")
labels_edited  = np.load("E:/STUDY/Publications/Thesis/Brain Emotion Detection/Dataset/Practice with datasets/Seed/de_LDS_Extracted/Shuffled X y/shuffled_labels.npy")

In [3]:
max = np.max(data)
min = np.min(data)

#Normalizing data
data = data/max
data = (data - np.mean(data))/np.std(data)


#Splitting Dataset into train, validation, test 
train_labels = labels_edited[0:550]
test_labels = labels_edited[550:600]
final_labels = labels_edited[600:675]

train_data = data[0:550]
test_data = data[550:600]
final_test = data[600:675]



cf_labels = np.where(final_labels== -1, 2, final_labels) #Label in the form for confusion matrix
un, co = np.unique(cf_labels, return_counts=True)
print(f'Unique: {un}, Counts: {co}')

Unique: [0. 1. 2.], Counts: [25 25 25]


In [13]:
train_labels_reshaped = train_labels.reshape(-1,1) #Formatting for input to the CNN model
test_labels_reshaped = test_labels.reshape(-1,1)
final_labels_reshaped = final_labels.reshape(-1,1)

train_labels_reshaped = to_categorical(train_labels_reshaped, 3) #One Hot Encoding
test_labels_reshaped = to_categorical(test_labels_reshaped, 3)
final_labels_reshaped = to_categorical(final_labels_reshaped, 3)

# Load the CNN model and extract

In [11]:
from keras.models import load_model, Model

# Load the model
model = load_model(r'E:\STUDY\Publications\Thesis\Brain Emotion Detection\Code Practice\Conf 1\version 14\model.h5')
intermediate_layer_model = Model(inputs=model.input, outputs=model.output)

# # Feature extraction using the provided model
train_features = intermediate_layer_model.predict(train_data)
test_features = intermediate_layer_model.predict(test_data)

# train_features = Flatten()(train_features)
# val_features = Flatten()(val_features)
# test_features = Flatten()(test_features)

2/2 [==============================] - 1s 355ms/step


In [15]:
from keras.models import load_model

# Load the model
model = load_model(r'E:\STUDY\Publications\Thesis\Brain Emotion Detection\Code Practice\Conf 1\version 14\model.h5')



In [17]:
# Evaluate the model on the training data
evaluation_result_train = model.evaluate(train_data, train_labels_reshaped, verbose=0)
train_loss = evaluation_result_train[0]
train_accuracy = evaluation_result_train[1]

print(f'Training Accuracy: {train_accuracy * 100:.2f}%')

# Evaluate the model on the test data
evaluation_result_test = model.evaluate(final_test, final_labels_reshaped, verbose=0)
test_loss = evaluation_result_test[0]
test_accuracy = evaluation_result_test[1]

print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

Training Accuracy: 95.82%
Test Accuracy: 89.33%


In [6]:
train_features = model.predict(train_data)
test_features = model.predict(final_test)

3/3 [==============================] - 2s 463ms/step


In [7]:
scaler = StandardScaler()
train_features_std = scaler.fit_transform(train_features)
test_features_std = scaler.transform(test_features)

In [8]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

# Create a voting ensemble with uninitialized classifiers
ensemble_model = VotingClassifier(estimators=[
    ('svm', SVC(probability=True, random_state=42)),
    ('knn', KNeighborsClassifier()),
    ('rf', RandomForestClassifier(random_state=42)),
    ('dt', DecisionTreeClassifier(random_state=42)),
], voting='soft')  # 'soft' for probability voting

# Train the ensemble model
ensemble_model.fit(train_features_std, train_labels)

# Evaluate on the test set
ensemble_test_preds = ensemble_model.predict(test_features_std)
test_accuracy = accuracy_score(final_labels, ensemble_test_preds)
print('Ensemble Test Accuracy:', test_accuracy)

# Check if the ensemble accuracy is greater than 92
if test_accuracy > 0.92:
    print("Ensemble model with accuracy greater than 92 is trained.")
else:
    print("Ensemble model does not meet the desired accuracy threshold.")

Ensemble Test Accuracy: 0.9333333333333333
Ensemble model with accuracy greater than 92 is trained.
